## 1. Import libraries

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [3]:
import pandas as pd
import csv
import time
import re
import tqdm.notebook as tq
from tqdm.notebook import tqdm

## 2. Prepare file csv for crawl

In [20]:
# collect movies name and overview, release date from previous dataset
csv_file_path1 = "F:/DS_Projects/Latest10000movies/dataset/Movies_dataset.csv"
csv_file_path2 = "F:/DS_Projects/Latest10000movies/top10K_TMDB_movies.csv"
csv_file_path3 = "F:/DS_Projects/Latest10000movies/moviesofcredits.csv"

In [22]:
# read csv by pandas
df1 = pd.read_csv(csv_file_path1)
df2 = pd.read_csv(csv_file_path2)
df3 = pd.read_csv(csv_file_path3)

In [23]:
df1

,index,title,original_language,release_date,popularity,vote_average,vote_count,overview
0,0,Spider-Man: Across the Spider-Verse,en,31-05-2023,3368.627,8.5,3386,"After reuniting with Gwen Stacy, Brooklyn’s fu..."
1,1,Transformers: Rise of the Beasts,en,06-06-2023,2160.316,7.5,2747,When a new threat capable of destroying the en...
2,2,The Flash,en,13-06-2023,2108.713,7.0,2305,When his attempt to save his family inadverten...
3,3,No Hard Feelings,en,15-06-2023,1978.358,7.1,562,"On the brink of losing her childhood home, Mad..."
4,4,Barbie,en,19-07-2023,1976.513,7.5,3013,Barbie and Ken are having the time of their li...
...,...,...,...,...,...,...,...,...
9995,9995,Porky's II: The Next Day,en,24-06-1983,12.617,5.7,273,When the students of Angel Beach High decide t...
9996,9996,Nobody Knows,ja,07-08-2004,12.617,8.0,427,"In a small Tokyo apartment, twelve-year-old Ak..."
9997,9997,Rollerball,en,25-06-1975,12.615,6.2,443,"In a corporate-controlled future, an ultra-vio..."
9998,9998,On Golden Pond,en,01-12-1981,12.614,7.3,404,"For Norman and Ethel Thayer, this summer on go..."


In [26]:
# Drop unneccesary column
df1.drop(['index'],inplace = True,axis = 1)

In [27]:
df1.drop_duplicates()

,title,original_language,release_date,popularity,vote_average,vote_count,overview
0,Spider-Man: Across the Spider-Verse,en,31-05-2023,3368.627,8.5,3386,"After reuniting with Gwen Stacy, Brooklyn’s fu..."
1,Transformers: Rise of the Beasts,en,06-06-2023,2160.316,7.5,2747,When a new threat capable of destroying the en...
2,The Flash,en,13-06-2023,2108.713,7.0,2305,When his attempt to save his family inadverten...
3,No Hard Feelings,en,15-06-2023,1978.358,7.1,562,"On the brink of losing her childhood home, Mad..."
4,Barbie,en,19-07-2023,1976.513,7.5,3013,Barbie and Ken are having the time of their li...
...,...,...,...,...,...,...,...
9995,Porky's II: The Next Day,en,24-06-1983,12.617,5.7,273,When the students of Angel Beach High decide t...
9996,Nobody Knows,ja,07-08-2004,12.617,8.0,427,"In a small Tokyo apartment, twelve-year-old Ak..."
9997,Rollerball,en,25-06-1975,12.615,6.2,443,"In a corporate-controlled future, an ultra-vio..."
9998,On Golden Pond,en,01-12-1981,12.614,7.3,404,"For Norman and Ethel Thayer, this summer on go..."


In [24]:
df2

,id,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count
0,278,The Shawshank Redemption,"Drama,Crime",en,Framed in the 1940s for the double murder of h...,94.075,1994-09-23,8.7,21862
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance",hi,"Raj is a rich, carefree, happy-go-lucky second...",25.408,1995-10-19,8.7,3731
2,238,The Godfather,"Drama,Crime",en,"Spanning the years 1945 to 1955, a chronicle o...",90.585,1972-03-14,8.7,16280
3,424,Schindler's List,"Drama,History,War",en,The true story of how businessman Oskar Schind...,44.761,1993-12-15,8.6,12959
4,240,The Godfather: Part II,"Drama,Crime",en,In the continuing saga of the Corleone crime f...,57.749,1974-12-20,8.6,9811
...,...,...,...,...,...,...,...,...,...
9995,10196,The Last Airbender,"Action,Adventure,Fantasy",en,"The story follows the adventures of Aang, a yo...",98.322,2010-06-30,4.7,3347
9996,331446,Sharknado 3: Oh Hell No!,"Action,TV Movie,Science Fiction,Comedy,Adventure",en,The sharks take bite out of the East Coast whe...,12.490,2015-07-22,4.7,417
9997,13995,Captain America,"Action,Science Fiction,War",en,"During World War II, a brave, patriotic Americ...",18.333,1990-12-14,4.6,332
9998,2312,In the Name of the King: A Dungeon Siege Tale,"Adventure,Fantasy,Action,Drama",en,A man named Farmer sets out to rescue his kidn...,15.159,2007-11-29,4.7,668


In [29]:
# Drop unneccesary column
df2.drop(['id', 'genre'],inplace = True,axis = 1)

In [30]:
df2.drop_duplicates()

,title,original_language,overview,popularity,release_date,vote_average,vote_count
0,The Shawshank Redemption,en,Framed in the 1940s for the double murder of h...,94.075,1994-09-23,8.7,21862
1,Dilwale Dulhania Le Jayenge,hi,"Raj is a rich, carefree, happy-go-lucky second...",25.408,1995-10-19,8.7,3731
2,The Godfather,en,"Spanning the years 1945 to 1955, a chronicle o...",90.585,1972-03-14,8.7,16280
3,Schindler's List,en,The true story of how businessman Oskar Schind...,44.761,1993-12-15,8.6,12959
4,The Godfather: Part II,en,In the continuing saga of the Corleone crime f...,57.749,1974-12-20,8.6,9811
...,...,...,...,...,...,...,...
9995,The Last Airbender,en,"The story follows the adventures of Aang, a yo...",98.322,2010-06-30,4.7,3347
9996,Sharknado 3: Oh Hell No!,en,The sharks take bite out of the East Coast whe...,12.490,2015-07-22,4.7,417
9997,Captain America,en,"During World War II, a brave, patriotic Americ...",18.333,1990-12-14,4.6,332
9998,In the Name of the King: A Dungeon Siege Tale,en,A man named Farmer sets out to rescue his kidn...,15.159,2007-11-29,4.7,668


In [31]:
# rename answer column
df2.columns = ['title', 'original_language', 'release_date', 'popularity', 'vote_average', 'vote_count', 'overview']

In [32]:
df2

,title,original_language,release_date,popularity,vote_average,vote_count,overview
0,The Shawshank Redemption,en,Framed in the 1940s for the double murder of h...,94.075,1994-09-23,8.7,21862
1,Dilwale Dulhania Le Jayenge,hi,"Raj is a rich, carefree, happy-go-lucky second...",25.408,1995-10-19,8.7,3731
2,The Godfather,en,"Spanning the years 1945 to 1955, a chronicle o...",90.585,1972-03-14,8.7,16280
3,Schindler's List,en,The true story of how businessman Oskar Schind...,44.761,1993-12-15,8.6,12959
4,The Godfather: Part II,en,In the continuing saga of the Corleone crime f...,57.749,1974-12-20,8.6,9811
...,...,...,...,...,...,...,...
9995,The Last Airbender,en,"The story follows the adventures of Aang, a yo...",98.322,2010-06-30,4.7,3347
9996,Sharknado 3: Oh Hell No!,en,The sharks take bite out of the East Coast whe...,12.490,2015-07-22,4.7,417
9997,Captain America,en,"During World War II, a brave, patriotic Americ...",18.333,1990-12-14,4.6,332
9998,In the Name of the King: A Dungeon Siege Tale,en,A man named Farmer sets out to rescue his kidn...,15.159,2007-11-29,4.7,668


In [33]:
df2['release_date'], df2['vote_average'], df2['vote_count'], df2['overview'] = df2['vote_average'], df2['vote_count'], df2['overview'], df2['release_date']

In [34]:
df2

,title,original_language,release_date,popularity,vote_average,vote_count,overview
0,The Shawshank Redemption,en,1994-09-23,94.075,8.7,21862,Framed in the 1940s for the double murder of h...
1,Dilwale Dulhania Le Jayenge,hi,1995-10-19,25.408,8.7,3731,"Raj is a rich, carefree, happy-go-lucky second..."
2,The Godfather,en,1972-03-14,90.585,8.7,16280,"Spanning the years 1945 to 1955, a chronicle o..."
3,Schindler's List,en,1993-12-15,44.761,8.6,12959,The true story of how businessman Oskar Schind...
4,The Godfather: Part II,en,1974-12-20,57.749,8.6,9811,In the continuing saga of the Corleone crime f...
...,...,...,...,...,...,...,...
9995,The Last Airbender,en,2010-06-30,98.322,4.7,3347,"The story follows the adventures of Aang, a yo..."
9996,Sharknado 3: Oh Hell No!,en,2015-07-22,12.490,4.7,417,The sharks take bite out of the East Coast whe...
9997,Captain America,en,1990-12-14,18.333,4.6,332,"During World War II, a brave, patriotic Americ..."
9998,In the Name of the King: A Dungeon Siege Tale,en,2007-11-29,15.159,4.7,668,A man named Farmer sets out to rescue his kidn...


In [35]:
# Convert the 'date' column to datetime format
df2['release_date'] = pd.to_datetime(df2['release_date'])

# Convert the datetime column to the desired format
df2['release_date'] = df2['release_date'].dt.strftime('%d-%m-%Y')

In [36]:
df2

,title,original_language,release_date,popularity,vote_average,vote_count,overview
0,The Shawshank Redemption,en,23-09-1994,94.075,8.7,21862,Framed in the 1940s for the double murder of h...
1,Dilwale Dulhania Le Jayenge,hi,19-10-1995,25.408,8.7,3731,"Raj is a rich, carefree, happy-go-lucky second..."
2,The Godfather,en,14-03-1972,90.585,8.7,16280,"Spanning the years 1945 to 1955, a chronicle o..."
3,Schindler's List,en,15-12-1993,44.761,8.6,12959,The true story of how businessman Oskar Schind...
4,The Godfather: Part II,en,20-12-1974,57.749,8.6,9811,In the continuing saga of the Corleone crime f...
...,...,...,...,...,...,...,...
9995,The Last Airbender,en,30-06-2010,98.322,4.7,3347,"The story follows the adventures of Aang, a yo..."
9996,Sharknado 3: Oh Hell No!,en,22-07-2015,12.490,4.7,417,The sharks take bite out of the East Coast whe...
9997,Captain America,en,14-12-1990,18.333,4.6,332,"During World War II, a brave, patriotic Americ..."
9998,In the Name of the King: A Dungeon Siege Tale,en,29-11-2007,15.159,4.7,668,A man named Farmer sets out to rescue his kidn...


In [25]:
df3

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",10-12-2009,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",19-05-2007,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",26-10-2015,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",16-07-2012,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",07-03-2012,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",04-09-1992,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],26-12-2011,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

In [37]:
# Drop unneccesary column
df3.drop(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_title', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline'],inplace = True,axis = 1)

In [40]:
df3

,original_language,overview,popularity,release_date,title,vote_average,vote_count
0,en,"In the 22nd century, a paraplegic Marine is di...",150.437577,10-12-2009,Avatar,7.2,11800
1,en,"Captain Barbossa, long believed to be dead, ha...",139.082615,19-05-2007,Pirates of the Caribbean: At World's End,6.9,4500
2,en,A cryptic message from Bond’s past sends him o...,107.376788,26-10-2015,Spectre,6.3,4466
3,en,Following the death of District Attorney Harve...,112.312950,16-07-2012,The Dark Knight Rises,7.6,9106
4,en,"John Carter is a war-weary, former military ca...",43.926995,07-03-2012,John Carter,6.1,2124
...,...,...,...,...,...,...,...
4798,es,El Mariachi just wants to play his guitar and ...,14.269792,04-09-1992,El Mariachi,6.6,238
4799,en,A newlywed couple's honeymoon is upended by th...,0.642552,26-12-2011,Newlyweds,5.9,5
4800,en,"""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,13-10-2013,"Signed, Sealed, Delivered",7.0,6
4801,en,When ambitious New York attorney Sam is sent t...,0.857008,03-05-2012,Shanghai Calling,5.7,7


In [39]:
df3.drop(['production_companies', 'production_countries'],inplace = True,axis = 1)

In [41]:
df3.drop_duplicates()

,original_language,overview,popularity,release_date,title,vote_average,vote_count
0,en,"In the 22nd century, a paraplegic Marine is di...",150.437577,10-12-2009,Avatar,7.2,11800
1,en,"Captain Barbossa, long believed to be dead, ha...",139.082615,19-05-2007,Pirates of the Caribbean: At World's End,6.9,4500
2,en,A cryptic message from Bond’s past sends him o...,107.376788,26-10-2015,Spectre,6.3,4466
3,en,Following the death of District Attorney Harve...,112.312950,16-07-2012,The Dark Knight Rises,7.6,9106
4,en,"John Carter is a war-weary, former military ca...",43.926995,07-03-2012,John Carter,6.1,2124
...,...,...,...,...,...,...,...
4798,es,El Mariachi just wants to play his guitar and ...,14.269792,04-09-1992,El Mariachi,6.6,238
4799,en,A newlywed couple's honeymoon is upended by th...,0.642552,26-12-2011,Newlyweds,5.9,5
4800,en,"""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,13-10-2013,"Signed, Sealed, Delivered",7.0,6
4801,en,When ambitious New York attorney Sam is sent t...,0.857008,03-05-2012,Shanghai Calling,5.7,7


In [42]:
# rename answer column
df3.columns = ['title', 'original_language', 'release_date', 'popularity', 'vote_average', 'vote_count', 'overview']
df3

,title,original_language,release_date,popularity,vote_average,vote_count,overview
0,en,"In the 22nd century, a paraplegic Marine is di...",150.437577,10-12-2009,Avatar,7.2,11800
1,en,"Captain Barbossa, long believed to be dead, ha...",139.082615,19-05-2007,Pirates of the Caribbean: At World's End,6.9,4500
2,en,A cryptic message from Bond’s past sends him o...,107.376788,26-10-2015,Spectre,6.3,4466
3,en,Following the death of District Attorney Harve...,112.312950,16-07-2012,The Dark Knight Rises,7.6,9106
4,en,"John Carter is a war-weary, former military ca...",43.926995,07-03-2012,John Carter,6.1,2124
...,...,...,...,...,...,...,...
4798,es,El Mariachi just wants to play his guitar and ...,14.269792,04-09-1992,El Mariachi,6.6,238
4799,en,A newlywed couple's honeymoon is upended by th...,0.642552,26-12-2011,Newlyweds,5.9,5
4800,en,"""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,13-10-2013,"Signed, Sealed, Delivered",7.0,6
4801,en,When ambitious New York attorney Sam is sent t...,0.857008,03-05-2012,Shanghai Calling,5.7,7


In [43]:
df3['title'], df3['original_language'], df3['release_date'], df3['popularity'], df3['vote_average'], df3['vote_count'], df3['overview'] = df3['vote_average'], df3['title'], df3['popularity'], df3['release_date'], df3['vote_count'], df3['overview'], df3['original_language']
df3

,title,original_language,release_date,popularity,vote_average,vote_count,overview
0,Avatar,en,10-12-2009,150.437577,7.2,11800,"In the 22nd century, a paraplegic Marine is di..."
1,Pirates of the Caribbean: At World's End,en,19-05-2007,139.082615,6.9,4500,"Captain Barbossa, long believed to be dead, ha..."
2,Spectre,en,26-10-2015,107.376788,6.3,4466,A cryptic message from Bond’s past sends him o...
3,The Dark Knight Rises,en,16-07-2012,112.312950,7.6,9106,Following the death of District Attorney Harve...
4,John Carter,en,07-03-2012,43.926995,6.1,2124,"John Carter is a war-weary, former military ca..."
...,...,...,...,...,...,...,...
4798,El Mariachi,es,04-09-1992,14.269792,6.6,238,El Mariachi just wants to play his guitar and ...
4799,Newlyweds,en,26-12-2011,0.642552,5.9,5,A newlywed couple's honeymoon is upended by th...
4800,"Signed, Sealed, Delivered",en,13-10-2013,1.444476,7.0,6,"""Signed, Sealed, Delivered"" introduces a dedic..."
4801,Shanghai Calling,en,03-05-2012,0.857008,5.7,7,When ambitious New York attorney Sam is sent t...


In [44]:
# Append rows from df2 to df1 where df1 does not have the same rows
df = pd.concat([df1, df2], ignore_index=True).drop_duplicates()
df

,title,original_language,release_date,popularity,vote_average,vote_count,overview
0,Spider-Man: Across the Spider-Verse,en,31-05-2023,3368.627,8.5,3386,"After reuniting with Gwen Stacy, Brooklyn’s fu..."
1,Transformers: Rise of the Beasts,en,06-06-2023,2160.316,7.5,2747,When a new threat capable of destroying the en...
2,The Flash,en,13-06-2023,2108.713,7.0,2305,When his attempt to save his family inadverten...
3,No Hard Feelings,en,15-06-2023,1978.358,7.1,562,"On the brink of losing her childhood home, Mad..."
4,Barbie,en,19-07-2023,1976.513,7.5,3013,Barbie and Ken are having the time of their li...
...,...,...,...,...,...,...,...
19995,The Last Airbender,en,30-06-2010,98.322,4.7,3347,"The story follows the adventures of Aang, a yo..."
19996,Sharknado 3: Oh Hell No!,en,22-07-2015,12.490,4.7,417,The sharks take bite out of the East Coast whe...
19997,Captain America,en,14-12-1990,18.333,4.6,332,"During World War II, a brave, patriotic Americ..."
19998,In the Name of the King: A Dungeon Siege Tale,en,29-11-2007,15.159,4.7,668,A man named Farmer sets out to rescue his kidn...


In [46]:
# Append rows from df2 to df1 where df1 does not have the same rows
df = pd.concat([df, df3], ignore_index=True).drop_duplicates()
df

,title,original_language,release_date,popularity,vote_average,vote_count,overview
0,Spider-Man: Across the Spider-Verse,en,31-05-2023,3368.627000,8.5,3386,"After reuniting with Gwen Stacy, Brooklyn’s fu..."
1,Transformers: Rise of the Beasts,en,06-06-2023,2160.316000,7.5,2747,When a new threat capable of destroying the en...
2,The Flash,en,13-06-2023,2108.713000,7.0,2305,When his attempt to save his family inadverten...
3,No Hard Feelings,en,15-06-2023,1978.358000,7.1,562,"On the brink of losing her childhood home, Mad..."
4,Barbie,en,19-07-2023,1976.513000,7.5,3013,Barbie and Ken are having the time of their li...
...,...,...,...,...,...,...,...
24364,El Mariachi,es,04-09-1992,14.269792,6.6,238,El Mariachi just wants to play his guitar and ...
24365,Newlyweds,en,26-12-2011,0.642552,5.9,5,A newlywed couple's honeymoon is upended by th...
24366,"Signed, Sealed, Delivered",en,13-10-2013,1.444476,7.0,6,"""Signed, Sealed, Delivered"" introduces a dedic..."
24367,Shanghai Calling,en,03-05-2012,0.857008,5.7,7,When ambitious New York attorney Sam is sent t...


In [47]:
df.reset_index(inplace=True)
df

,index,title,original_language,release_date,popularity,vote_average,vote_count,overview
0,0,Spider-Man: Across the Spider-Verse,en,31-05-2023,3368.627000,8.5,3386,"After reuniting with Gwen Stacy, Brooklyn’s fu..."
1,1,Transformers: Rise of the Beasts,en,06-06-2023,2160.316000,7.5,2747,When a new threat capable of destroying the en...
2,2,The Flash,en,13-06-2023,2108.713000,7.0,2305,When his attempt to save his family inadverten...
3,3,No Hard Feelings,en,15-06-2023,1978.358000,7.1,562,"On the brink of losing her childhood home, Mad..."
4,4,Barbie,en,19-07-2023,1976.513000,7.5,3013,Barbie and Ken are having the time of their li...
...,...,...,...,...,...,...,...,...
24364,24364,El Mariachi,es,04-09-1992,14.269792,6.6,238,El Mariachi just wants to play his guitar and ...
24365,24365,Newlyweds,en,26-12-2011,0.642552,5.9,5,A newlywed couple's honeymoon is upended by th...
24366,24366,"Signed, Sealed, Delivered",en,13-10-2013,1.444476,7.0,6,"""Signed, Sealed, Delivered"" introduces a dedic..."
24367,24367,Shanghai Calling,en,03-05-2012,0.857008,5.7,7,When ambitious New York attorney Sam is sent t...


In [48]:
df.to_csv("F:/DS_Projects/Latest10000movies/dataset/24kMoviesDataset.csv")

In [5]:
csv_file_path = "F:/DS_Projects/Movies/dataset/24kMoviesDataset.csv"

In [6]:
df = pd.read_csv(csv_file_path)
df

,Unnamed: 0,index,title,original_language,release_date,popularity,vote_average,vote_count,overview
0,0,0,Spider-Man: Across the Spider-Verse,en,31-05-2023,3368.627000,8.5,3386,"After reuniting with Gwen Stacy, Brooklyn’s fu..."
1,1,1,Transformers: Rise of the Beasts,en,06-06-2023,2160.316000,7.5,2747,When a new threat capable of destroying the en...
2,2,2,The Flash,en,13-06-2023,2108.713000,7.0,2305,When his attempt to save his family inadverten...
3,3,3,No Hard Feelings,en,15-06-2023,1978.358000,7.1,562,"On the brink of losing her childhood home, Mad..."
4,4,4,Barbie,en,19-07-2023,1976.513000,7.5,3013,Barbie and Ken are having the time of their li...
...,...,...,...,...,...,...,...,...,...
24364,24364,24364,El Mariachi,es,04-09-1992,14.269792,6.6,238,El Mariachi just wants to play his guitar and ...
24365,24365,24365,Newlyweds,en,26-12-2011,0.642552,5.9,5,A newlywed couple's honeymoon is upended by th...
24366,24366,24366,"Signed, Sealed, Delivered",en,13-10-2013,1.444476,7.0,6,"""Signed, Sealed, Delivered"" introduces a dedic..."
24367,24367,24367,Shanghai Calling,en,03-05-2012,0.857008,5.7,7,When ambitious New York attorney Sam is sent t...


In [7]:
df.drop(['Unnamed: 0'],inplace = True,axis = 1)

In [8]:
df

,index,title,original_language,release_date,popularity,vote_average,vote_count,overview
0,0,Spider-Man: Across the Spider-Verse,en,31-05-2023,3368.627000,8.5,3386,"After reuniting with Gwen Stacy, Brooklyn’s fu..."
1,1,Transformers: Rise of the Beasts,en,06-06-2023,2160.316000,7.5,2747,When a new threat capable of destroying the en...
2,2,The Flash,en,13-06-2023,2108.713000,7.0,2305,When his attempt to save his family inadverten...
3,3,No Hard Feelings,en,15-06-2023,1978.358000,7.1,562,"On the brink of losing her childhood home, Mad..."
4,4,Barbie,en,19-07-2023,1976.513000,7.5,3013,Barbie and Ken are having the time of their li...
...,...,...,...,...,...,...,...,...
24364,24364,El Mariachi,es,04-09-1992,14.269792,6.6,238,El Mariachi just wants to play his guitar and ...
24365,24365,Newlyweds,en,26-12-2011,0.642552,5.9,5,A newlywed couple's honeymoon is upended by th...
24366,24366,"Signed, Sealed, Delivered",en,13-10-2013,1.444476,7.0,6,"""Signed, Sealed, Delivered"" introduces a dedic..."
24367,24367,Shanghai Calling,en,03-05-2012,0.857008,5.7,7,When ambitious New York attorney Sam is sent t...


In [9]:
duplicate = df[df.duplicated(['title', 'release_date', 'overview'])]
print("Duplicate Rows based on title :")
 
# Print the resultant Dataframe
duplicate

Duplicate Rows based on title :


,index,title,original_language,release_date,popularity,vote_average,vote_count,overview
9566,9566,The Shawshank Redemption,en,23-09-1994,94.075000,8.7,21862,Framed in the 1940s for the double murder of h...
9568,9568,The Godfather,en,14-03-1972,90.585000,8.7,16280,"Spanning the years 1945 to 1955, a chronicle o..."
9569,9569,Schindler's List,en,15-12-1993,44.761000,8.6,12959,The true story of how businessman Oskar Schind...
9571,9571,Impossible Things,es,17-06-2021,14.358000,8.6,255,"Matilde is a woman who, after the death of her..."
9572,9572,Spirited Away,ja,20-07-2001,92.056000,8.5,13093,"A young girl, Chihiro, becomes trapped in a st..."
...,...,...,...,...,...,...,...,...
24289,24289,The Gallows,en,10-07-2015,18.045782,4.9,361,20 years after a horrific accident during a sm...
24296,24296,Cheap Thrills,en,08-03-2013,10.826253,6.3,155,"Recently fired and facing eviction, a new dad ..."
24324,24324,The Signal,en,15-03-2014,27.662696,5.8,631,Three college students on a road trip across t...
24339,24339,Clerks,en,13-09-1994,19.748658,7.4,755,Convenience and video store clerks Dante and R...


In [10]:
df_res = df.drop_duplicates(subset=['title', 'release_date', 'overview'])

In [11]:
df_res

,index,title,original_language,release_date,popularity,vote_average,vote_count,overview
0,0,Spider-Man: Across the Spider-Verse,en,31-05-2023,3368.627000,8.5,3386,"After reuniting with Gwen Stacy, Brooklyn’s fu..."
1,1,Transformers: Rise of the Beasts,en,06-06-2023,2160.316000,7.5,2747,When a new threat capable of destroying the en...
2,2,The Flash,en,13-06-2023,2108.713000,7.0,2305,When his attempt to save his family inadverten...
3,3,No Hard Feelings,en,15-06-2023,1978.358000,7.1,562,"On the brink of losing her childhood home, Mad..."
4,4,Barbie,en,19-07-2023,1976.513000,7.5,3013,Barbie and Ken are having the time of their li...
...,...,...,...,...,...,...,...,...
24364,24364,El Mariachi,es,04-09-1992,14.269792,6.6,238,El Mariachi just wants to play his guitar and ...
24365,24365,Newlyweds,en,26-12-2011,0.642552,5.9,5,A newlywed couple's honeymoon is upended by th...
24366,24366,"Signed, Sealed, Delivered",en,13-10-2013,1.444476,7.0,6,"""Signed, Sealed, Delivered"" introduces a dedic..."
24367,24367,Shanghai Calling,en,03-05-2012,0.857008,5.7,7,When ambitious New York attorney Sam is sent t...


In [15]:
df_res.drop(['index'],inplace = True,axis = 1)

C:\Users\dell\AppData\Local\Temp\ipykernel_2944\2179562553.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_res.drop(['index'],inplace = True,axis = 1)


In [16]:
df_res

,title,original_language,release_date,popularity,vote_average,vote_count,overview
0,Spider-Man: Across the Spider-Verse,en,31-05-2023,3368.627000,8.5,3386,"After reuniting with Gwen Stacy, Brooklyn’s fu..."
1,Transformers: Rise of the Beasts,en,06-06-2023,2160.316000,7.5,2747,When a new threat capable of destroying the en...
2,The Flash,en,13-06-2023,2108.713000,7.0,2305,When his attempt to save his family inadverten...
3,No Hard Feelings,en,15-06-2023,1978.358000,7.1,562,"On the brink of losing her childhood home, Mad..."
4,Barbie,en,19-07-2023,1976.513000,7.5,3013,Barbie and Ken are having the time of their li...
...,...,...,...,...,...,...,...
24364,El Mariachi,es,04-09-1992,14.269792,6.6,238,El Mariachi just wants to play his guitar and ...
24365,Newlyweds,en,26-12-2011,0.642552,5.9,5,A newlywed couple's honeymoon is upended by th...
24366,"Signed, Sealed, Delivered",en,13-10-2013,1.444476,7.0,6,"""Signed, Sealed, Delivered"" introduces a dedic..."
24367,Shanghai Calling,en,03-05-2012,0.857008,5.7,7,When ambitious New York attorney Sam is sent t...


In [21]:
# Reset the index to match row numbers
df_res.reset_index(drop=True, inplace=True)

In [23]:
# Save the merged DataFrame back to the CSV file
df_res.to_csv("F:/DS_Projects/Movies/dataset/17kMoviesDataset.csv", index=True, index_label='index')

## 2. Crawl Data

In [4]:
def check_overview(str1, str2):
    return str1[:30] == str2[:30]

In [5]:
# Define the regular expression pattern
pattern = r"\/movie\/(\d+)[\-\?]"
# function to collect movie id through url
def collect_url(url_str):
    # Use re.search to find the match
    match = re.search(pattern, url_str)

    # Check if a match is found
    if match:
        # Extract the matched number group (group 1)
        movie_number = match.group(1)
        return movie_number
    else:
        return " "

In [6]:
def extract_values(soup, idx, movie_id):
    # Initialize a dictionary to store the extracted values
    extracted_values = {}
    
    extracted_values["index"] = idx
    extracted_values["movie_id"] = movie_id
    
    # find movie's name
    movie_name = soup.find('h2').find('a').text.strip()
    extracted_values["title"] = movie_name

    # find genres
    genres_span = soup.find(class_="genres")
    if genres_span:
        # Extract the text, remove non-breaking spaces, and split into a list
        # Now genres_text contains a list of genres
        genres = genres_span.get_text(strip=True).replace('\xa0', '').split(',')
        # convert list to a string for easier write to file and dataframe
        extracted_values["genres"] = ', '.join(genres)
    else:
        extracted_values["genres"] = " "


    # find certification
    certification = soup.find(class_="certification")
    if certification:
        # Now certifiaction return a string contain certification
        extracted_values["Certification"] = certification.get_text(strip=True)
    else:
        extracted_values["Certification"] = " "

    
    # find runtime
    runtime = soup.find(class_="runtime")
    if runtime:
        # Now runtime return a string contain runtime
        extracted_values["Runtime"] = runtime.get_text(strip=True)
    else:
        extracted_values["Runtime"] = " "
        
        
    # find status, language, budget, revenue.
    p_tags = soup.find_all('p')

    # Iterate through the <p> tags 
    for p_tag in p_tags:
        strong_bdi = p_tag.find('strong')
        if strong_bdi:
            bdi = strong_bdi.find('bdi')
            if bdi:
                key = bdi.get_text()
                value = p_tag.get_text(strip=True).replace(key, '').strip()
                extracted_values[key] = value
    
    
    # find directors & writers
    directors = []
    writers = []
    # Find all <li> tags with class "profile"
    profile_items = soup.find_all('li', class_='profile')

    if profile_items:
        # Iterate through the <li> tags and extract information
        for item in profile_items:
            name = item.find('a').text
            role = item.find('p', class_='character').text
            if "Director" in role:
                directors.append(name)
            if "Writer" in role or "Screenplay" in role:
                writers.append(name)
            
        extracted_values["directors"] = ', '.join(directors)
        extracted_values["writers"] = ', '.join(writers)
    else:
        extracted_values["directors"] = ' '
        extracted_values["writers"] = ' '
            
    # find actors
    actors = []
    # Find all <li> tags with class "profile"
    card_items = soup.find_all('li', class_='card')
    
    if card_items:
        # Iterate through the <li> tags and extract information
        for item in card_items:
            name = item.find('p').find('a').text.strip()
            actors.append(name)
    
        extracted_values["actors"] = ', '.join(actors)
    else:
        extracted_values["actors"] = ' '
    
    
    # find keywords
    keywords = []
    # Find the <section> element with class "keywords"
    section_element = soup.find('section', class_='keywords right_column')

    if section_element.find('ul'):
        # Find the <ul> element within the <section>
        ul_element = section_element.find('ul')

        # Find all <li> elements within the <ul>
    
        li_elements = ul_element.find_all('li')

        # Iterate through the <li> elements and extract the text
        for li in li_elements:
            keyword = li.text.strip()  # Remove leading/trailing whitespace
            keywords.append(keyword)
        
        extracted_values["keywords"] = ', '.join(keywords)
    else:
        extracted_values["keywords"] = " "
    
    return extracted_values

In [7]:
# url to the movie database website
# just a random movie
URL = "https://www.themoviedb.org/movie/64690-drive?language=en"

## 3. test run with a single movie

In [24]:
movie_name = "Game of Death"

In [25]:
# function to go to the movie's page
driver = webdriver.Chrome()
try:
    driver.get(URL)

except Exception as e:
    print(f"An error occurred: {e}")

In [26]:
# crawl test for a single movie
contents = None
try:   
    search_button = driver.find_element(By.CLASS_NAME, 'search')
    search_button.click()

    search_input = driver.find_element(By.CLASS_NAME, 'k-input')
    search_input.clear()  
    search_input.send_keys(movie_name)
     
    # Pressing enter to search input text
    actions = ActionChains(driver)
    actions.send_keys(Keys.ENTER)
    actions.perform()
    
    # find only movies
#     movies_tab = driver.find_element(By.ID, "movie")  
#     movies_tab.click()
#     search_result = driver.find_element(By.CSS_SELECTOR, 'a[data-media-type="movie"]')
    
#     # Click the element
#     search_result.click()
    
    # New code to click an element with a specific description
    desired_overview = "A martial arts movie star must fake his death to find the people who are trying to kill him."  # Change this to your desired date March 23, 1978
    
    overview_elements = driver.find_elements(By.CLASS_NAME, "overview")
    
    # Find all div elements with class "title"
    title_elements = driver.find_elements(By.CLASS_NAME, "title")
    
    for i in range(len(overview_elements)):
        overview_element = overview_elements[i]
        title_element = title_elements[i]

        # Check if the release date matches the desired date
        if check_overview(overview_element.text.strip(), desired_overview):
            # Find the `a` tag within the `title` div
            print(overview_element.text.strip())
            a_tag = title_element.find_element(By.TAG_NAME, "a")
            
            #driver.execute_script("arguments[0].scrollIntoView();", a_tag)
            
            # Wait for the input element to become invisible or disabled
#             wait = WebDriverWait(driver, 10)
#             wait.until(EC.invisibility_of_element_located((By.ID, "search_v4")))

            # Click the `a` tag
            a_tag.click()
            break  # Exit the loop after clicking the first matching element
            
    html = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
    soup = BeautifulSoup(html, "html.parser")
    
    get_url = driver.current_url
    url_number = collect_url(get_url)
    
    contents = extract_values(soup, 1, url_number)
    
    #time.sleep(1)
except Exception as e:
    print(f"An error occurred: {e}")

A martial arts movie star must fake his death to find the people who are trying to kill him.


In [28]:
# Close the browser after you're done
driver.quit()

In [27]:
for key, value in contents.items():
    print(key, ": ", value)

index :  1
movie_id :  13333
title :  Game of Death
genres :  Drama, Action, Thriller
Certification :  IIB
Runtime :  1h 41m
Status :  Released
Original Language :  Cantonese
Budget :  $850,000.00
Revenue :  -
directors :  Robert Clouse
writers :  Robert Clouse
actors :  Bruce Lee, Gig Young, Dean Jagger, Hugh O'Brian, Colleen Camp, Robert Wall, Mel Novak, Kareem Abdul-Jabbar, Chuck Norris
keywords :  martial arts, sword fight, revenge, fighting, east asian lead, one against many, action hero


## 4. Crawl the whole dataset

In [4]:
csv_crawl_path = "F:/DS_Projects/Movies/dataset/15kMoviesDataset.csv"

In [5]:
crawl_df = pd.read_csv(csv_crawl_path)
crawl_df

,index,title,original_language,release_date,popularity,vote_average,vote_count,overview
0,0,Spider-Man: Across the Spider-Verse,en,31-05-2023,3368.627000,8.5,3386,"After reuniting with Gwen Stacy, Brooklyn’s fu..."
1,1,Transformers: Rise of the Beasts,en,06-06-2023,2160.316000,7.5,2747,When a new threat capable of destroying the en...
2,2,The Flash,en,13-06-2023,2108.713000,7.0,2305,When his attempt to save his family inadverten...
3,3,No Hard Feelings,en,15-06-2023,1978.358000,7.1,562,"On the brink of losing her childhood home, Mad..."
4,4,Barbie,en,19-07-2023,1976.513000,7.5,3013,Barbie and Ken are having the time of their li...
...,...,...,...,...,...,...,...,...
15917,15917,Cavite,en,12-03-2005,0.022173,7.5,2,"Adam, a security guard, travels from Californi..."
15918,15918,Newlyweds,en,26-12-2011,0.642552,5.9,5,A newlywed couple's honeymoon is upended by th...
15919,15919,"Signed, Sealed, Delivered",en,13-10-2013,1.444476,7.0,6,"""Signed, Sealed, Delivered"" introduces a dedic..."
15920,15920,Shanghai Calling,en,03-05-2012,0.857008,5.7,7,When ambitious New York attorney Sam is sent t...


In [15]:
movies_data = []

In [16]:
# function to go to the movie's page
driver = webdriver.Chrome()

try:
    driver.get(URL)
except Exception as e:
    print(f"An error occurred: {e}")

In [17]:
movie_data = None
movie_name = None
# loop for file
for i in tqdm(range(len(crawl_df["title"]))):
    try:
        search_button = driver.find_element(By.CLASS_NAME, 'search')
        search_button.click()
        
        movie_name = crawl_df["title"].iloc[i]
        search_input = driver.find_element(By.CLASS_NAME, 'k-input')
        search_input.clear()
        search_input.send_keys(movie_name)
     
        # Pressing enter to search input text
        actions = ActionChains(driver)
        actions.send_keys(Keys.ENTER)
        actions.perform()
        
        # find only movies
        movies_tab = driver.find_element(By.ID, "movie")  
        movies_tab.click()
    
    
        # Code to click an element with a specific description
        desired_overview = crawl_df["overview"].iloc[i]    
        overview_elements = driver.find_elements(By.CLASS_NAME, "overview")
    
        # Find all div elements with class "title"
        title_elements = driver.find_elements(By.CLASS_NAME, "title")
    
        for j in range(len(overview_elements)):
            overview_element = overview_elements[j]
            title_element = title_elements[j]

            if overview_element:
                # Check if the overview matches the desired overview
                if check_overview(overview_element.text.strip(), desired_overview):
                    # Find the `a` tag within the `title` div
                    a_tag = title_element.find_element(By.TAG_NAME, "a")
            
                    # Click the `a` tag
                    a_tag.click()
                    
                    # collect html content
                    html = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
                    soup = BeautifulSoup(html, "html.parser")
                    
                    get_url = driver.current_url
                    url_number = collect_url(get_url)
        
                    # extract and collect data
                    movie_data = extract_values(soup, crawl_df["index"].iloc[i], url_number)
                    movies_data.append(movie_data)
                    
                    break  # Exit the loop after clicking the first matching element
                    
            else:
                search_result = driver.find_element(By.CSS_SELECTOR, 'a[data-media-type="movie"]')
    
                # Click the first element
                search_result.click()
                html = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
                soup = BeautifulSoup(html, "html.parser")
                
                # get movie id
                get_url = driver.current_url
                url_number = collect_url(get_url)
        
                # extract and collect data
                movie_data = extract_values(soup, crawl_df["index"].iloc[i], url_number)
                movies_data.append(movie_data)
                    
                break  # Exit the loop after clicking the first matching element
                    
        #print(f"Received idx: {i}")
        
    except Exception as e:
        print(f"An error occurred: {e} at the movie: {movie_name}, index: {i}")
        
# Close the browser
driver.quit()

  0%|          | 0/15922 [00:00<?, ?it/s]

An error occurred: 'float' object is not subscriptable at the movie: Los Habitantes, index: 56
An error occurred: 'float' object is not subscriptable at the movie: El hotel de los líos. García y García 2, index: 107
An error occurred: 'float' object is not subscriptable at the movie: Los bastardos, index: 147
An error occurred: 'float' object is not subscriptable at the movie: Doble discurso, index: 225
An error occurred: 'float' object is not subscriptable at the movie: Gabriel's Inferno: Part IV, index: 280
An error occurred: 'float' object is not subscriptable at the movie: Za gyakutai: Nyotai ikedori-hen, index: 553
An error occurred: 'float' object is not subscriptable at the movie: Proof of Love, index: 588
An error occurred: 'float' object is not subscriptable at the movie: Si todas las puertas se cierran, index: 785
An error occurred: 'float' object is not subscriptable at the movie: El Lugar de Nora, index: 1249
An error occurred: 'float' object is not subscriptable at the mov

An error occurred: 'float' object is not subscriptable at the movie: King of Snipers, index: 6985
An error occurred: 'float' object is not subscriptable at the movie: Papa, index: 7095
An error occurred: 'float' object is not subscriptable at the movie: Memórias da Chuva, index: 7376
An error occurred: 'float' object is not subscriptable at the movie: Lincessa. Los silencios del bosque, index: 7378
An error occurred: 'float' object is not subscriptable at the movie: Red Bull Dragon Ball FighterZ World Final Paris, index: 7742
An error occurred: 'float' object is not subscriptable at the movie: Attraction 3, index: 7866
An error occurred: 'float' object is not subscriptable at the movie: Sound of Freedom, index: 7900
An error occurred: 'float' object is not subscriptable at the movie: Mais Pesado é o Céu, index: 7951
An error occurred: 'float' object is not subscriptable at the movie: K: Seven Stories Movie 6 - Circle Vision - Nameless Song, index: 7986
An error occurred: 'float' object

In [13]:
# Close the browser
driver.quit()

In [18]:
movies_df = pd.DataFrame(movies_data)
movies_df

,index,movie_id,title,genres,Certification,Runtime,Status,Original Language,Budget,Revenue,directors,writers,actors,keywords
0,0,569094,Spider-Man: Across the Spider-Verse,"Animation, Action, Adventure",PG,2h 20m,Released,English,"$100,000,000.00","$683,241,751.00","Joaquim Dos Santos, Justin K. Thompson, Kemp P...","Phil Lord, Christopher Miller, Dave Callaham","Shameik Moore, Hailee Steinfeld, Jason Schwart...","sacrifice, comic book, sequel, superhero, base..."
1,1,667538,Transformers: Rise of the Beasts,"Action, Adventure, Science Fiction",PG-13,2h 7m,Released,English,"$195,000,000.00","$429,800,000.00",Steven Caple Jr.,"Joby Harold, Jon Hoeber, Erich Hoeber, Darnell...","Anthony Ramos, Dominique Fishback, Peter Culle...","peru, alien, end of the world, based on toy, r..."
2,2,298618,The Flash,"Action, Adventure, Science Fiction",PG-13,2h 24m,Released,English,"$300,000,000.00","$268,375,616.00",Andy Muschietti,Christina Hodson,"Ezra Miller, Sasha Calle, Michael Keaton, Mich...","secret identity, hero, superhero, based on com..."
3,3,884605,No Hard Feelings,"Comedy, Romance",R,1h 44m,Released,English,"$45,000,000.00","$82,995,526.00",Gene Stupnitsky,"Gene Stupnitsky, John Phillips","Jennifer Lawrence, Andrew Barth Feldman, Laura...","romantic comedy, summer, new york state, homet..."
4,4,346698,Barbie,"Comedy, Adventure, Fantasy",PG-13,1h 54m,Released,English,"$145,000,000.00","$1,427,900,000.00",Greta Gerwig,"Greta Gerwig, Noah Baumbach","Margot Robbie, Ryan Gosling, America Ferrera, ...","feminism, patriarchy, satire, based on toy, fe..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14840,15915,286939,Sanctuary; Quite a Conundrum,"Thriller, Horror, Comedy",NR,1h 22m,Released,English,-,-,Thomas L. Phillips,Thomas L. Phillips,"Sasha Ramos, Erin Cline, Emily Rogers, Anthony...",
14841,15916,124606,Bang,Drama,,1h 38m,Released,English,-,-,Ash Baron-Cohen,Ash Baron-Cohen,"Darling Narita, Peter Greene, Michael Newland,...","gang, audition, police fake, homeless"
14842,15918,72766,Newlyweds,"Comedy, Romance, Drama",,1h 25m,Released,English,"$9,000.00","$5,000.00",Edward Burns,Edward Burns,"Edward Burns, Caitlin FitzGerald, Kerry Bishé,...",
14843,15920,126186,Shanghai Calling,"Romance, Comedy, Drama",,1h 38m,Released,Chinese,-,-,Daniel Hsia,Daniel Hsia,"Daniel Henney, Eliza Coupe, Bill Paxton, Alan ...",


In [19]:
movies_df.to_csv("F:/DS_Projects/Movies/dataset/Scrape15k.csv",index = False)

In [7]:
csv_result_path = "F:/DS_Projects/Movies/dataset/Scrape15k.csv"

In [8]:
result_df = pd.read_csv(csv_result_path)
result_df

,index,movie_id,title,genres,Certification,Runtime,Status,Original Language,Budget,Revenue,directors,writers,actors,keywords
0,0,569094,Spider-Man: Across the Spider-Verse,"Animation, Action, Adventure",PG,2h 20m,Released,English,"$100,000,000.00","$683,241,751.00","Joaquim Dos Santos, Justin K. Thompson, Kemp P...","Phil Lord, Christopher Miller, Dave Callaham","Shameik Moore, Hailee Steinfeld, Jason Schwart...","sacrifice, comic book, sequel, superhero, base..."
1,1,667538,Transformers: Rise of the Beasts,"Action, Adventure, Science Fiction",PG-13,2h 7m,Released,English,"$195,000,000.00","$429,800,000.00",Steven Caple Jr.,"Joby Harold, Jon Hoeber, Erich Hoeber, Darnell...","Anthony Ramos, Dominique Fishback, Peter Culle...","peru, alien, end of the world, based on toy, r..."
2,2,298618,The Flash,"Action, Adventure, Science Fiction",PG-13,2h 24m,Released,English,"$300,000,000.00","$268,375,616.00",Andy Muschietti,Christina Hodson,"Ezra Miller, Sasha Calle, Michael Keaton, Mich...","secret identity, hero, superhero, based on com..."
3,3,884605,No Hard Feelings,"Comedy, Romance",R,1h 44m,Released,English,"$45,000,000.00","$82,995,526.00",Gene Stupnitsky,"Gene Stupnitsky, John Phillips","Jennifer Lawrence, Andrew Barth Feldman, Laura...","romantic comedy, summer, new york state, homet..."
4,4,346698,Barbie,"Comedy, Adventure, Fantasy",PG-13,1h 54m,Released,English,"$145,000,000.00","$1,427,900,000.00",Greta Gerwig,"Greta Gerwig, Noah Baumbach","Margot Robbie, Ryan Gosling, America Ferrera, ...","feminism, patriarchy, satire, based on toy, fe..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14840,15915,286939,Sanctuary; Quite a Conundrum,"Thriller, Horror, Comedy",NR,1h 22m,Released,English,-,-,Thomas L. Phillips,Thomas L. Phillips,"Sasha Ramos, Erin Cline, Emily Rogers, Anthony...",
14841,15916,124606,Bang,Drama,,1h 38m,Released,English,-,-,Ash Baron-Cohen,Ash Baron-Cohen,"Darling Narita, Peter Greene, Michael Newland,...","gang, audition, police fake, homeless"
14842,15918,72766,Newlyweds,"Comedy, Romance, Drama",,1h 25m,Released,English,"$9,000.00","$5,000.00",Edward Burns,Edward Burns,"Edward Burns, Caitlin FitzGerald, Kerry Bishé,...",
14843,15920,126186,Shanghai Calling,"Romance, Comedy, Drama",,1h 38m,Released,Chinese,-,-,Daniel Hsia,Daniel Hsia,"Daniel Henney, Eliza Coupe, Bill Paxton, Alan ...",


In [9]:
merged_df = pd.merge(crawl_df, result_df, left_on=['index','title'], right_on=['index','title'], how='left')
merged_df

,index,title,original_language,release_date,popularity,vote_average,vote_count,overview,movie_id,genres,Certification,Runtime,Status,Original Language,Budget,Revenue,directors,writers,actors,keywords
0,0,Spider-Man: Across the Spider-Verse,en,31-05-2023,3368.627000,8.5,3386,"After reuniting with Gwen Stacy, Brooklyn’s fu...",569094.0,"Animation, Action, Adventure",PG,2h 20m,Released,English,"$100,000,000.00","$683,241,751.00","Joaquim Dos Santos, Justin K. Thompson, Kemp P...","Phil Lord, Christopher Miller, Dave Callaham","Shameik Moore, Hailee Steinfeld, Jason Schwart...","sacrifice, comic book, sequel, superhero, base..."
1,1,Transformers: Rise of the Beasts,en,06-06-2023,2160.316000,7.5,2747,When a new threat capable of destroying the en...,667538.0,"Action, Adventure, Science Fiction",PG-13,2h 7m,Released,English,"$195,000,000.00","$429,800,000.00",Steven Caple Jr.,"Joby Harold, Jon Hoeber, Erich Hoeber, Darnell...","Anthony Ramos, Dominique Fishback, Peter Culle...","peru, alien, end of the world, based on toy, r..."
2,2,The Flash,en,13-06-2023,2108.713000,7.0,2305,When his attempt to save his family inadverten...,298618.0,"Action, Adventure, Science Fiction",PG-13,2h 24m,Released,English,"$300,000,000.00","$268,375,616.00",Andy Muschietti,Christina Hodson,"Ezra Miller, Sasha Calle, Michael Keaton, Mich...","secret identity, hero, superhero, based on com..."
3,3,No Hard Feelings,en,15-06-2023,1978.358000,7.1,562,"On the brink of losing her childhood home, Mad...",884605.0,"Comedy, Romance",R,1h 44m,Released,English,"$45,000,000.00","$82,995,526.00",Gene Stupnitsky,"Gene Stupnitsky, John Phillips","Jennifer Lawrence, Andrew Barth Feldman, Laura...","romantic comedy, summer, new york state, homet..."
4,4,Barbie,en,19-07-2023,1976.513000,7.5,3013,Barbie and Ken are having the time of their li...,346698.0,"Comedy, Adventure, Fantasy",PG-13,1h 54m,Released,English,"$145,000,000.00","$1,427,900,000.00",Greta Gerwig,"Greta Gerwig, Noah Baumbach","Margot Robbie, Ryan Gosling, America Ferrera, ...","feminism, patriarchy, satire, based on toy, fe..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15917,15917,Cavite,en,12-03-2005,0.022173,7.5,2,"Adam, a security guard, travels from Californi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15918,15918,Newlyweds,en,26-12-2011,0.642552,5.9,5,A newlywed couple's honeymoon is upended by th...,72766.0,"Comedy, Romance, Drama",,1h 25m,Released,English,"$9,000.00","$5,000.00",Edward Burns,Edward Burns,"Edward Burns, Caitlin FitzGerald, Kerry Bishé,...",
15919,15919,"Signed, Sealed, Delivered",en,13-10-2013,1.444476,7.0,6,"""Signed, Sealed, Delivered"" introduces a dedic...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15920,15920,Shanghai Calling,en,03-05-2012,0.857008,5.7,7,When ambitious New York attorney Sam is sent t...,126186.0,"Romance, Comedy, Drama",,1h 38m,Released,Chinese,-,-,Daniel Hsia,Daniel Hsia,"Daniel Henney, Eliza Coupe, Bill Paxton, Alan ...",


In [10]:
# Save the merged DataFrame back to the CSV file
merged_df.to_csv("F:/DS_Projects/Movies/dataset/MoviesDataset.csv",index = False)

## transform

In [39]:
# Function to convert time format to minutes
def convert_time_to_minutes(time_str):
    # Use regular expressions to extract hours and minutes
    match = re.match(r'(\d+)h(\d+)?m?', time_str)
    
    if re.match(r'(\d+)h(\d+)?m?', time_str):
        hours = int(match.group(1)) if match.group(1) else 0
        minutes = int(match.group(2)) if match.group(2) else 0
        
        total_minutes = hours * 60 + minutes
            
        return total_minutes
    elif re.match(r'(\d+)?m?', time_str):
        minutes = int(re.match(r'(\d+)?m?', time_str).group(1))  # Extract minutes directly
        return minutes
    else:
        return None

# Example usage
time_str1 = "2h30m"
time_str2 = "2h "
time_str3 = "90m"

minutes1 = convert_time_to_minutes(time_str1)
minutes2 = convert_time_to_minutes(time_str2)
minutes3 = convert_time_to_minutes(time_str3)

print(minutes1)
print(minutes2)
print(minutes3)

150
120
90
